In [120]:
CLIENT_ID = "doZap8T-60t8-8SsgzKPxg"
# Add secret key
SECRET_KEY  = ""

In [121]:
with open('pwd.txt', 'r') as file:
    pw = file.read()

# Getting Access To Reddit API

In [122]:
import praw
reddit = praw.Reddit(client_id = CLIENT_ID,
                     client_secret = SECRET_KEY,
                     username = 'Carlosshadow5',
                     password = pw,
                     user_agent = 'NPL_Project')

# Getting Data by Search

In [123]:
import pandas as pd

# Gets the subreddit
subreddit = reddit.subreddit("BigFive")

## Neuroticism Posts

In [124]:
neuroticism_data = {}
neuroticism_df = pd.DataFrame(neuroticism_data)
neuroticism_user_names = []
neuroticism_title = []
neuroticism_text = []

subreddit_search = subreddit.search("neuroticism", limit = None)
for post in subreddit_search:
    neuroticism_user_names.append(post.author.name)
    neuroticism_title.append(post.title)
    neuroticism_text.append(post.selftext)

neuroticism_df['User Name'] = neuroticism_user_names
neuroticism_df['Post Title'] = neuroticism_title
neuroticism_df['Post Text'] = neuroticism_text
neuroticism_df

,User Name,Post Title,Post Text
0,noneedtothinktomuch,how do I deal with extreme neuroticism combine...,My mind seems to try to commit suicide when th...
1,SirOlimusDesferalPAX,How to cope with very low neuroticism combined...,"As an example, I have to take a very important..."
2,SirOlimusDesferalPAX,How to deal with very low neuroticism?,As to be motivated by a possibility of a negat...
3,jesstheloudfan44,Just took a test. Anything I can to to increas...,
4,waynefan77991,"Any advice for someone of average openness, lo...","I recently took the Understand Myself Test, an..."
...,...,...,...
200,cldu1,I am constantly very anxious but never angry o...,What does it mean for neuroticism?\n\nSo I get...
201,xsaon,What can you tell about me from my score?,Hey everyone :) So I did the IPIP-NEO 300 ques...
202,lpetrich,Extremes of Big Five traits?,"Of the Big Five traits, one end of their spect..."
203,RTX2080Ti0,Big Five + AB5C + PatternTypes + Correlations,I am 25 70 100 70 00.\n\nSetup: NN EE OO AA CC...


## Extraversion Posts

In [125]:
extraversion_data = {}
extraversion_df = pd.DataFrame(extraversion_data)
extraversion_user_names = []
extraversion_title = []
extraversion_text = []

subreddit_search = subreddit.search("extraversion", limit = None)
for post in subreddit_search:
    extraversion_user_names.append(post.author.name)
    extraversion_title.append(post.title)
    extraversion_text.append(post.selftext)

extraversion_df['User Name'] = extraversion_user_names
extraversion_df['Post Title'] = extraversion_title
extraversion_df['Post Text'] = extraversion_text
extraversion_df

,User Name,Post Title,Post Text
0,Krisington22,"If I just get my extraversion up, I could go f...",
1,UnfeelingGamer,Anyone else have similar results? Agreeablenes...,Here are my results - what do you think this t...
2,wjsgus95,Big 5 Results: Extremely Low Openness & Extrav...,Hi I'm a 26 year old software engineer \n\nbec...
3,SlateAlmond90,What is the difference between the Warmth and ...,
4,ThePopulousMishmash,It appears that I have managed to lower my agr...,"As far as I can assess, my brain works solely ..."
...,...,...,...
130,Fallen1729,Antinatalism,I recently came across the odd philosophy of a...
131,RTX2080Ti0,New way to explain your B5 results,**I have:**\n\n99 Openness\n\n1 Conscientiousn...
132,lpetrich,US Presidents,"I've found the book ""Personality, Character, a..."
133,lpetrich,Personality Dimensions in Nonhuman Animals: A ...,"""In a recent article in the Los Angeles Times,..."


## Openness Posts

In [126]:
openness_data = {}
openness_df = pd.DataFrame(openness_data)
openness_user_names = []
openness_title = []
openness_text = []

subreddit_search = subreddit.search("openness", limit = None)
for post in subreddit_search:
    openness_user_names.append(post.author.name)
    openness_title.append(post.title)
    openness_text.append(post.selftext)

openness_df['User Name'] = openness_user_names
openness_df['Post Title'] = openness_title
openness_df['Post Text'] = openness_text
openness_df

,User Name,Post Title,Post Text
0,waynefan77991,I'm not high in openness but wish I was,"I don't know how high I am in openness, I've t..."
1,jesstheloudfan44,Just took a test. Anything I can to to increas...,
2,mrafee113,I think my openness is probably less than my r...,Scored at 98th%tile. I think it's actually les...
3,RobertLee2354,"Hi everyone, I consistently score 100% on Open...",
4,UnfeelingGamer,Anyone else have similar results? Agreeablenes...,Here are my results - what do you think this t...
...,...,...,...
203,fintechSGNYC,Can someone who is low in neuroticism still ge...,And if yes would the symptoms or experiences i...
204,RTX2080Ti0,New way to explain your B5 results,**I have:**\n\n99 Openness\n\n1 Conscientiousn...
205,ItzFin,Could people maybe stop viewing some traits as...,I've seen a trend in this sub where a lot of p...
206,lpetrich,US Presidents,"I've found the book ""Personality, Character, a..."


## Agreeableness Posts

In [127]:
agreeableness_data = {}
agreeableness_df = pd.DataFrame(agreeableness_data)
agreeableness_user_names = []
agreeableness_title = []
agreeableness_text = []

subreddit_search = subreddit.search("agreeableness", limit = None)
for post in subreddit_search:
    agreeableness_user_names.append(post.author.name)
    agreeableness_title.append(post.title)
    agreeableness_text.append(post.selftext)

agreeableness_df['User Name'] = agreeableness_user_names
agreeableness_df['Post Title'] = agreeableness_title
agreeableness_df['Post Text'] = agreeableness_text
agreeableness_df

,User Name,Post Title,Post Text
0,jesstheloudfan44,Just took a test. Anything I can to to increas...,
1,Dependent-Sherbet-94,What causes someone to be low or high on agree...,
2,Dependent-Sherbet-94,Low Agreeableness and Intimate Relationships,I recently read on a book that once people get...
3,vampiric-weasel12,Do autistic people usually have low agreeablen...,\n\n[View Poll](https://www.reddit.com/poll/10...
4,Dravonishere,Mostly accurate. Explains why I’ve been in man...,
...,...,...,...
186,RTX2080Ti0,New way to explain your B5 results,**I have:**\n\n99 Openness\n\n1 Conscientiousn...
187,lpetrich,Non-Pejorative Names for the Five Factors?,From the Gosling-John review of animal persona...
188,lpetrich,US Presidents,"I've found the book ""Personality, Character, a..."
189,lpetrich,Personality Dimensions in Nonhuman Animals: A ...,"""In a recent article in the Los Angeles Times,..."


## Conscientiousness Posts

In [128]:
conscientiousness_data = {}
conscientiousness_df = pd.DataFrame(conscientiousness_data)
conscientiousness_user_names = []
conscientiousness_title = []
conscientiousness_text = []

subreddit_search = subreddit.search("conscientiousness", limit = None)
for post in subreddit_search:
    conscientiousness_user_names.append(post.author.name)
    conscientiousness_title.append(post.title)
    conscientiousness_text.append(post.selftext)

conscientiousness_df['User Name'] = conscientiousness_user_names
conscientiousness_df['Post Title'] = conscientiousness_title
conscientiousness_df['Post Text'] = conscientiousness_text
conscientiousness_df

,User Name,Post Title,Post Text
0,noneedtothinktomuch,how do I deal with extreme neuroticism combine...,My mind seems to try to commit suicide when th...
1,SirOlimusDesferalPAX,How to increase conscientiousness?,Professionally diagnosed IQ 3SD above & neglec...
2,MyManSteveBuscemi,Does Conscientiousness cause certain negative ...,"Nobody can seem to agree on whether the ""disgu..."
3,Existing-Persimmon32,Genomelink came really close to accurately pre...,
4,waynefan77991,"Any advice for someone of average openness, lo...","I recently took the Understand Myself Test, an..."
...,...,...,...
181,gibigigorgegebman,what does depression look like on the big 5?,does agreeableness go down? I know conscientio...
182,RTX2080Ti0,New way to explain your B5 results,**I have:**\n\n99 Openness\n\n1 Conscientiousn...
183,lpetrich,US Presidents,"I've found the book ""Personality, Character, a..."
184,lpetrich,Non-Pejorative Names for the Five Factors?,From the Gosling-John review of animal persona...


# Get Post From Username

In [130]:
# Get username 
username = "lpetrich"

user = reddit.redditor(username)
submissions = user.submissions.new(limit = None)

for post in submissions:
    # Use post.title for title
    # Use post.selftext for text
    print(post.title)

Uncle Bernie translations: Tío Bernie, Amo Bernie, ...
Ranked-pairs algorithms?
Computational performance of vote-counting algorithms: How fast? How bulky?
Should US State Senates be abolished?
Latin American monarchs
Nineteenth-century European nation builders wanted monarchies
Republics to monarchies?
How Monarchies End
Why are the British royals such big celebrities?
Monarchical Republics?
Lindsay Holiday: Which European monarchies are next to fall?
Demographics: where do you live (more non-UK)?
Are monarchies more stable than republics?
What AOC might say to Kyrsten Sinema about Build Back Better
From exposure to lead: less conscientious, less agreeable, and more neurotic
US Presidents' Big Five Personalities
The Big Five vs. the Four Temperaments
The Big Five vs. the Enneagram
The Big Five vs. DISC
The Big Five vs. MBTI
Supertraits of the Big Five
The Evolution of the Big Five Factors
The Big Five factors in our brains
The Big Five, HEXACO, and the Dark Triad
Comparing the Big Fiv

# Average Posts per User

In [153]:
from prawcore.exceptions import Forbidden
def getUniqueUsernames(all_usernames):
    unique_usernames = []

    for user in all_usernames:
        if user not in unique_usernames:
            unique_usernames.append(user)
    
    return unique_usernames

all_usernames =  neuroticism_user_names + extraversion_user_names + openness_user_names + agreeableness_user_names + conscientiousness_user_names
unique_usernames = getUniqueUsernames(all_usernames)

total_posts = 0

print(f'Total number of unique users: {len(unique_usernames)}')

for user in unique_usernames:
    # Get their reddit user profile
    reddit_user = reddit.redditor(user)
    submissions = reddit_user.submissions.new(limit = None)
    try:
        for post in submissions:
            total_posts += 1
    except Forbidden:
        print("Cant")
    
        

average_posts_per_user = total_posts/len(unique_usernames)

print(f'The average posts per user is {average_posts_per_user}')


Total number of unique users: 254
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
